In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
class DenseCNN(nn.Module):
    def __init__(self, num_classes):
        super(DenseCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernal_size=5)
        self.dropout = nn.Dropout(0.5)

        self.conv2 = nn.Conv2d(64, 128, kernal_size=3)
        self.dropout2 = nn.Dropout(0.5)

        self.conv3 = nn.Conv2d(128, 256, kernal_size=5)
        self.dropout = nn.Dropout(0.5)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)

        # Calculate the input size for the first fully connected layer
        self.fc1 = nn.Linear(512 * 60 * 24, 1024)  # Adjust based on pooling and input size
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = self.pool(nn.ReLU()(self.conv3(x)))
        x = self.pool(nn.ReLU()(self.conv4(x)))

        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(nn.ReLU()(self.fc1(x)))
        x = self.fc2(x)

        return x

In [3]:
# Hyperparameters
num_classes = 88  # Set the number of classes based on your dataset
learning_rate = 0.001
batch_size = 32
num_epochs = 10

In [4]:
transform = transforms.Compose([
    transforms.Resize((960, 369)),
    transforms.ToTensor(),
])


In [6]:
train_dataset = datasets.ImageFolder(root='data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [7]:
model = DenseCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
model.train()
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'dense_cnn_model.pth')

print("Training complete and model saved.")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x706560 and 737280x1024)